In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.font_manager import FontProperties

from utils.models import *

try:
    myfont = FontProperties(fname=r"/System/Library/Fonts/PingFang.ttc")
    sns.set(style="whitegrid", font=myfont.get_name())
except Exception as e:
    print(e)

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

dataA2 = pd.read_csv("../Version3/Data/A2.csv", low_memory=False)
dataA1 = pd.read_csv("../Version3/Data/A1.csv")

[Errno 2] No such file or directory: 'C:\\System\\Library\\Fonts\\PingFang.ttc'


In [2]:
car = pd.read_csv(os.path.join("../Version3/Data/DriverData/full_0.csv"), encoding='utf-8')
motor = pd.read_csv(os.path.join("../Version3/Data/DriverData/full_1.csv"), encoding='utf-8')
car_motor_overlap = pd.read_csv(os.path.join("../Version3/Data/DriverData/overlap_data.csv"), encoding='utf-8')
car_motor_out = pd.read_csv(os.path.join("../Version3/Data/DriverData/full_out.csv"), encoding='utf-8')

pass_0 = pd.read_csv(os.path.join("../Version3/Data/PassData/full_0.csv"), encoding='utf-8')
pass_1 = pd.read_csv(os.path.join("../Version3/Data/PassData/full_1.csv"), encoding='utf-8')
pass_out = pd.read_csv(os.path.join("../Version3/Data/PassData/full_out.csv"), encoding='utf-8')
pass_overlap = pd.read_csv(os.path.join("../Version3/Data/PassData/overlap_data.csv"), encoding='utf-8')

full_data = pd.read_csv(os.path.join("../Version3/Data/FullData/full_smaller.csv"), encoding='utf-8')

### 這是由拓樸得出來的特徵，可新增於拓樸訓練，但不增加在full_data

In [3]:
car_motor_out['type'] = 'out'
car_motor_overlap['type'] = 'overlap'
pass_out['type'] = 'out'
pass_overlap['type'] = 'overlap'

car_motor_out_overlap = pd.concat([car_motor_out, car_motor_overlap])

### 行人資料需要特別處理，因為他們沒有以下特徵

In [4]:
pass_0['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_0['當事者區分-類別-大類別名稱-車種'] = '人'
pass_1['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_1['當事者區分-類別-大類別名稱-車種'] = '人'
pass_out['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_out['當事者區分-類別-大類別名稱-車種'] = '人'
pass_overlap['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_overlap['當事者區分-類別-大類別名稱-車種'] = '人'
pass_out['行動電話或電腦或其他相類功能裝置名稱'] = '非駕駛人'
pass_out['當事者區分-類別-大類別名稱-車種'] = '人'

In [5]:
select_lst = [
    '天候名稱', '光線名稱', 
    '道路類別-第1當事者-名稱', '速限-第1當事者', 
    
    # 路面狀況
    '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱',
    # 號誌
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',
    # 車道
    '車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    # 當事人
    '當事者屬-性-別名稱', '當事者事故發生時年齡',
    '保護裝備名稱', '行動電話或電腦或其他相類功能裝置名稱',
    '肇事逃逸類別名稱-是否肇逃',

    # 大類別
    '道路型態大類別名稱', '事故位置大類別名稱',
    '車道劃分設施-分向設施大類別名稱',
    '事故類型及型態大類別名稱', '當事者區分-類別-大類別名稱-車種',
    '車輛撞擊部位大類別名稱-其他',
    '肇因研判大類別名稱-主要',

    # 子類別
    '道路型態子類別名稱', '事故位置子類別名稱', '事故類型及型態子類別名稱', '肇因研判子類別名稱-主要',
    '當事者區分-類別-子類別名稱-車種', '當事者行動狀態子類別名稱', '車輛撞擊部位子類別名稱-最初',
    '車輛撞擊部位子類別名稱-其他', '肇因研判子類別名稱-個別',
    
    '死亡'
]

# select data
car = car[select_lst]
motor = motor[select_lst]
pass_0['type'] = 'not overlap'
pass_1['type'] = 'not overlap'
passenger = pd.concat([pass_0, pass_1, pass_overlap])
passenger = passenger[select_lst]

full_data = full_data[select_lst]

select_lst.append('type')
car_motor_out_overlap = car_motor_out_overlap[select_lst]
pass_out = pass_out[select_lst]

## 唯一值檢查
第一區塊是完整模型的資料<br/>
第二區塊是多個模型個別進行模型，所以這裡需要量化後自動drop

In [6]:
print('詳細群體')
for data in [car, motor, car_motor_out_overlap, passenger, pass_out, full_data]:
    columns_to_drop = []
    for column in data.columns:
        if data[column].nunique() == 1:
            columns_to_drop.append(column)
    print(columns_to_drop)
    data.drop(columns=columns_to_drop, inplace=True)

詳細群體
['道路障礙-視距品質名稱', '道路障礙-視距名稱']
['道路障礙-視距品質名稱', '道路障礙-視距名稱']
[]
['道路障礙-視距品質名稱', '道路障礙-視距名稱', '行動電話或電腦或其他相類功能裝置名稱', '當事者區分-類別-大類別名稱-車種']
['行動電話或電腦或其他相類功能裝置名稱', '肇事逃逸類別名稱-是否肇逃', '當事者區分-類別-大類別名稱-車種', 'type']
[]


In [7]:
from sklearn.ensemble import RandomForestClassifier

def drop_low_importance_features(data, threshold=0.01):
    # One-hot 編碼資料
    X = pd.get_dummies(data.drop(columns=['死亡']))
    y = data['死亡']

    # 訓練 RandomForest 模型
    model = RandomForestClassifier()
    model.fit(X, y)

    # 計算特徵重要性
    feature_importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)

    # 篩選重要性高於 threshold 的 dummy 特徵
    selected_dummy_features = feature_importances[feature_importances > threshold].index.tolist()

    # 找回原始的類別特徵名稱
    original_features = data.drop(columns=['死亡']).columns
    selected_features = set()

    for feature in original_features:
        # 如果原始特徵的任何 dummy 特徵被選中，就保留原始特徵
        if any(dummy.startswith(f"{feature}_") for dummy in selected_dummy_features) or feature in selected_dummy_features:
            selected_features.add(feature)

    # 回到原始資料集中的選擇特徵
    reduced_data = data[list(selected_features) + ['死亡']]
    print(f"原始特徵數: {data.shape[1]}, 保留特徵數: {len(selected_features)}")
    return reduced_data

car_reduced = drop_low_importance_features(car)
motor_reduced = drop_low_importance_features(motor)
passenger_reduced = drop_low_importance_features(passenger)
car_motor_out_overlap_reduced = drop_low_importance_features(car_motor_out_overlap)
pass_out_reduced = drop_low_importance_features(pass_out)
full_data_reduced = drop_low_importance_features(full_data)

原始特徵數: 35, 保留特徵數: 14
原始特徵數: 35, 保留特徵數: 12
原始特徵數: 33, 保留特徵數: 17
原始特徵數: 38, 保留特徵數: 9
原始特徵數: 34, 保留特徵數: 20
原始特徵數: 37, 保留特徵數: 11


In [8]:
# car_0_dummy = pd.get_dummies(car)
# car_motor_out_overlap_dummy = pd.get_dummies(car_motor_out_overlap)
# motor_0_dummy = pd.get_dummies(motor)
# pass_0_dummy = pd.get_dummies(passenger)
# pass_out_dummy = pd.get_dummies(pass_out)

# full_data_dummy = pd.get_dummies(full_data)

car_0_dummy = pd.get_dummies(car_reduced)
car_motor_out_overlap_dummy = pd.get_dummies(car_motor_out_overlap_reduced)
motor_0_dummy = pd.get_dummies(motor_reduced)
pass_0_dummy = pd.get_dummies(passenger_reduced)
pass_out_dummy = pd.get_dummies(pass_out_reduced)

full_data_dummy = pd.get_dummies(full_data_reduced)

In [9]:
car_0_X, car_0_y = get_train_test_data(car_0_dummy)
motor_0_X, motor_0_y = get_train_test_data(motor_0_dummy)
car_motor_out_overlap_X, car_motor_out_overlap_y = get_train_test_data(car_motor_out_overlap_dummy)
pass_0_X, pass_0_y = get_train_test_data(pass_0_dummy)
pass_out_X, pass_out_y = get_train_test_data(pass_out_dummy)

full_data_X, full_data_y = get_train_test_data(full_data_dummy)

In [10]:
print(car_0_y.value_counts())
print(motor_0_y.value_counts())
print(car_motor_out_overlap_y.value_counts())
print(pass_0_y.value_counts())
print(pass_out_y.value_counts())

print(full_data_y.value_counts())

0    34237
1      151
Name: y, dtype: int64
0    56610
1      216
Name: y, dtype: int64
0    4588
1      57
Name: y, dtype: int64
0    2922
1      45
Name: y, dtype: int64
0    163
1     13
Name: y, dtype: int64
0    124382
1       550
Name: y, dtype: int64


In [11]:
print(car_0_X.shape)
print(motor_0_X.shape)
print(car_motor_out_overlap_X.shape)
print(pass_0_X.shape)
print(pass_out_X.shape)
print(full_data_X.shape)

(34388, 88)
(56826, 64)
(4645, 133)
(2967, 119)
(176, 131)
(124932, 77)


## 全部大類別資料kfold計算到motor0開始

In [12]:
import pickle
import time
import gc

models = [
    ("pass_0", pass_0_X, pass_0_y),
    ("pass_out", pass_out_X, pass_out_y),
    ("car_0", car_0_X, car_0_y),
    ("motor_0", motor_0_X, motor_0_y),
    ("car_motor_out_overlap", car_motor_out_overlap_X, car_motor_out_overlap_y),
    ("full_data", full_data_X, full_data_y),
]

# Logistic no fold
for name, X, y in models:
    print(f'{name} logistic start')
    start_time = time.time()
    # y_log, decision_scores_log, indices_log = logistic_cm_kfold(X.astype(float), y)
    y_log, decision_scores_log, indices_log = logistic_cm_gridsearch(X.astype(float), y)
    end_time = time.time()
    elapsed_time = end_time - start_time
    with open(f"../Models/ModelPerformanceNofoldImportance/logistic/{name}.pkl", "wb") as f:
        pickle.dump({
            'y': y_log,
            'decision_scores': decision_scores_log,
            'indices': indices_log,
            'elapsed_time': elapsed_time
        }, f)
    print(f'{name} logistic done in {elapsed_time:.2f} seconds')
    del X, y, y_log, decision_scores_log
    gc.collect()

# SVC no fold
for name, X, y in models:
    print(f'{name} svc start')
    start_time = time.time()
    y_svc, decision_scores_svc, indices_svc = linear_svc_cm_gridsearch(X.astype(float), y)
    end_time = time.time()
    elapsed_time = end_time - start_time
    with open(f"../Models/ModelPerformanceNofoldImportance/svc/{name}.pkl", "wb") as f:
        pickle.dump({
            'y': y_svc,
            'decision_scores': decision_scores_svc,
            'indices': indices_svc,
            'elapsed_time': elapsed_time
        }, f)
    print(f'{name} svc done in {elapsed_time:.2f} seconds')
    del X, y, y_svc, decision_scores_svc
    gc.collect()

# Logistic with fold
for name, X, y in models:
    print(f'{name} logistic start')
    start_time = time.time()
    y_log, decision_scores_log, indices_log = logistic_cm_kfold(X.astype(float), y)
    end_time = time.time()
    elapsed_time = end_time - start_time
    with open(f"../Models/ModelPerformanceImportance/logistic/{name}.pkl", "wb") as f:
        pickle.dump({
            'y': y_log,
            'decision_scores': decision_scores_log,
            'indices': indices_log,
            'elapsed_time': elapsed_time
        }, f)
    print(f'{name} logistic done in {elapsed_time:.2f} seconds')
    del X, y, y_log, decision_scores_log
    gc.collect()

# SVC with fold
for name, X, y in models:
    print(f'{name} svc start')
    start_time = time.time()
    y_svc, decision_scores_svc, indices_svc = linear_svc_kfold(X.astype(float), y, n_jobs=14)
    end_time = time.time()
    elapsed_time = end_time - start_time
    with open(f"../Models/ModelPerformanceImportance/svc/{name}.pkl", "wb") as f:
        pickle.dump({
            'y': y_svc,
            'decision_scores': decision_scores_svc,
            'indices': indices_svc,
            'elapsed_time': elapsed_time
        }, f)
    print(f'{name} svc done in {elapsed_time:.2f} seconds')
    del X, y, y_svc, decision_scores_svc
    gc.collect()

pass_0 logistic start
Best parameters found by GridSearchCV: {'C': 100, 'penalty': 'l1'}
pass_0 logistic done in 32.45 seconds
pass_out logistic start
Best parameters found by GridSearchCV: {'C': 100, 'penalty': 'l1'}
pass_out logistic done in 2.36 seconds
car_0 logistic start
Best parameters found by GridSearchCV: {'C': 100, 'penalty': 'l1'}
car_0 logistic done in 216.75 seconds
motor_0 logistic start
Best parameters found by GridSearchCV: {'C': 10, 'penalty': 'l2'}
motor_0 logistic done in 178.98 seconds
car_motor_out_overlap logistic start
Best parameters found by GridSearchCV: {'C': 100, 'penalty': 'l1'}
car_motor_out_overlap logistic done in 26.25 seconds
full_data logistic start
Best parameters found by GridSearchCV: {'C': 10, 'penalty': 'l2'}
full_data logistic done in 448.66 seconds
pass_0 svc start
Best parameters found by GridSearchCV: {'C': 10, 'loss': 'hinge'}
pass_0 svc done in 83.61 seconds
pass_out svc start
Best parameters found by GridSearchCV: {'C': 0.1, 'loss': 'hing

c:\Users\USER\anaconda3\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters found by GridSearchCV: {'C': 100, 'loss': 'hinge'}
car_0 svc done in 89.78 seconds
motor_0 svc start


c:\Users\USER\anaconda3\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters found by GridSearchCV: {'C': 10, 'loss': 'hinge'}
motor_0 svc done in 78.38 seconds
car_motor_out_overlap svc start


...

c:\Users\USER\anaconda3\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters for this fold: {'C': 100, 'loss': 'hinge'}


c:\Users\USER\anaconda3\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters for this fold: {'C': 10, 'loss': 'hinge'}
Best parameters for this fold: {'C': 100, 'loss': 'hinge'}
full_data svc done in 1635.91 seconds


c:\Users\USER\anaconda3\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
